# train

In [1]:
import yaml
import os

with open("./config.yaml", 'rb') as f:
    config = yaml.load(f)

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:5: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  """


In [2]:
OUTPUT_ROOT = config['IO_OPTION']['OUTPUT_ROOT']
os.makedirs(OUTPUT_ROOT, exist_ok=True)

## load library

In [3]:
# python default library
import os
import shutil
import datetime
import sys
import pickle

# general analysis tool-kit
import numpy as np
import pandas as pd
#from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import seaborn as sns

# pytorch
import torch
from torch import optim, nn
from torch.utils.tensorboard import SummaryWriter
import timm

#from torchvision.models import wide_resnet50_2, resnet18
#from torchsummary import summary

# etc
import yaml
yaml.warnings({'YAMLLoadWarning': False})

# original library
import common as com
import pytorch_modeler as modeler
from pytorch_model import EfficientNet_b1 as Model
#from pytorch_utils import filtered_load_model
#import models

import librosa
import IPython
import librosa.display

In [4]:
# effnet1 = timm.create_model('efficientnet_b1', pretrained=True)
# # M7:block[5], M8:block[6], M9:conv_head
# effnet1

## load config and set logger

In [5]:
log_folder = config['IO_OPTION']['OUTPUT_ROOT']+'/{0}.log'.format(datetime.date.today())
logger = com.setup_logger(log_folder, '00_train.py')

## Setting

In [6]:
# Setting seed
modeler.set_seed(42)

In [7]:
############################################################################
# Setting I/O path
############################################################################
# input dirs
INPUT_ROOT = config['IO_OPTION']['INPUT_ROOT']
dev_dir = INPUT_ROOT + "/dev_data"
add_dev_dir = INPUT_ROOT + "/add_dev_data"
# machine type
machine_types = os.listdir(dev_dir)
# output dirs
OUTPUT_ROOT = config['IO_OPTION']['OUTPUT_ROOT']
MODEL_DIR = config['IO_OPTION']['OUTPUT_ROOT'] + '/models'

TB_DIR = config['IO_OPTION']['OUTPUT_ROOT'] + '/tb'
OUT_FEATURE_DIR = OUTPUT_ROOT + '/extraction_features'
PRED_DIR = config['IO_OPTION']['OUTPUT_ROOT'] + '/pred'
SCORE_DIR = config['IO_OPTION']['OUTPUT_ROOT'] + '/score'
#os.makedirs(OUTPUT_ROOT, exist_ok=True)
os.makedirs(MODEL_DIR, exist_ok=True)
os.makedirs(TB_DIR, exist_ok=True)
os.makedirs(OUT_FEATURE_DIR, exist_ok=True)
os.makedirs(PRED_DIR, exist_ok=True)
os.makedirs(SCORE_DIR, exist_ok=True)

# copy config
# shutil.copy('./config.yaml', OUTPUT_ROOT)

In [8]:
torch.cuda.is_available()

True

## make path list

- __train_paths__ ([dict]): Each Machine Type
    - __ToyCar__ ([dict]): Each dataset
        - train ([list]) : paths
        - valid_source ([list]) : paths
        - valid_target ([list]) :paths
    - __ToyTrain__ ([dict]): Each dataset
        - train ([list]) : paths
        - valid_source ([list]) : paths
        - valid_target ([list]) : paths
    - ...

In [9]:
percent = 0.1

In [10]:
dev_paths = {}

for machine_type in machine_types:
    # dev train
    dev_train_paths = [f"{dev_dir}/{machine_type}/train/" + file for file in os.listdir(f"{dev_dir}/{machine_type}/train")]
    dev_train_paths = sorted(dev_train_paths)
    
    print(f'Machine {machine_type}', end=',')
    print(f'dev_train_size : {len(dev_train_paths)}', end=', ')
    dev_train_source_paths = [path for path in dev_train_paths if 'source' in path]
    print(f'source_size : {len(dev_train_source_paths)}', end=', ')
    dev_train_target_paths = [path for path in dev_train_paths if 'target' in path]
    print(f'target_size : {len(dev_train_target_paths)}', end=', ')
    # increse tgt
    num_increse_tgt = int(len(dev_train_source_paths) * percent)
    increse_dev_train_target_paths = dev_train_target_paths * num_increse_tgt
    # concat
    dev_train_paths = dev_train_source_paths + increse_dev_train_target_paths
    
    # add_dev train
    add_dev_paths = [f"{add_dev_dir}/{machine_type}/train/" + file for file in os.listdir(f"{add_dev_dir}/{machine_type}/train")]
    add_dev_paths = sorted(add_dev_paths)
    print(f'adddev_train_size : {len(add_dev_paths)}', end=', ')
    add_dev_source_paths = [path for path in add_dev_paths if 'source' in path]
    print(f'source_size : {len(add_dev_source_paths)}', end=', ')
    add_dev_target_paths = [path for path in add_dev_paths if 'target' in path]
    print(f'target_size : {len(add_dev_target_paths)}')
    # increse tgt
    num_increse_tgt = int(len(add_dev_source_paths) * percent)
    increse_add_dev_target_paths = add_dev_target_paths * num_increse_tgt
    # concat
    add_dev_paths = add_dev_source_paths + increse_add_dev_target_paths

    print('==============================================================')
    
    # dev_source valid
    dev_source_paths = [f"{dev_dir}/{machine_type}/source_test/" + file for file in os.listdir(f"{dev_dir}/{machine_type}/source_test")]
    dev_source_paths = sorted(dev_source_paths)
    
    # dev_target valid
    dev_target_paths = [f"{dev_dir}/{machine_type}/target_test/" + file for file in os.listdir(f"{dev_dir}/{machine_type}/target_test")]
    dev_target_paths = sorted(dev_target_paths)
    
    # bundle
    dev_paths[machine_type] = {}
    dev_paths[machine_type]['train'] = dev_train_paths + add_dev_paths
    dev_paths[machine_type]['valid_source'] = dev_source_paths
    dev_paths[machine_type]['valid_target'] = dev_target_paths

Machine pump,dev_train_size : 3009, source_size : 3000, target_size : 9, adddev_train_size : 3009, source_size : 3000, target_size : 9
Machine gearbox,dev_train_size : 3026, source_size : 3017, target_size : 9, adddev_train_size : 3114, source_size : 3105, target_size : 9
Machine slider,dev_train_size : 3009, source_size : 3000, target_size : 9, adddev_train_size : 3009, source_size : 3000, target_size : 9
Machine ToyCar,dev_train_size : 3009, source_size : 3000, target_size : 9, adddev_train_size : 3009, source_size : 3000, target_size : 9
Machine valve,dev_train_size : 3009, source_size : 3000, target_size : 9, adddev_train_size : 3009, source_size : 3000, target_size : 9
Machine ToyTrain,dev_train_size : 3009, source_size : 3000, target_size : 9, adddev_train_size : 3009, source_size : 3000, target_size : 9
Machine fan,dev_train_size : 3009, source_size : 3000, target_size : 9, adddev_train_size : 3009, source_size : 3000, target_size : 9


In [11]:
machine_types

['pump', 'gearbox', 'slider', 'ToyCar', 'valve', 'ToyTrain', 'fan']

In [12]:
#############################################################################
# run
#############################################################################
def run(machine_type, dev_paths):
    com.tic()
    os.makedirs(TB_DIR+'/'+machine_type, exist_ok=True)
    tb_log_dir = TB_DIR + '/' + machine_type
    writer = SummaryWriter(log_dir = tb_log_dir)
    
    logger.info('TARGET MACHINE_TYPE: {0}'.format(machine_type))
    logger.info('MAKE DATA_LOADER')
    # dev_train_paths
    dataloaders_dict = modeler.make_dataloader(dev_paths, machine_type)
    # parameter setting
    logger.info('TRAINING')
    model = Model(n_out=36, n_centers=6)
    
    optimizer = torch.optim.Adam(model.parameters(),lr=1e-3)
    output_dict, model, pred_df, score_df = modeler.run_training(model, dataloaders_dict, writer, optimizer)
    
    # output
    feature_out_path = f'{OUT_FEATURE_DIR}/{machine_type}_features.pkl'
    model_out_path = f'{MODEL_DIR}/{machine_type}_model.pkl'
    pred_out_path = f'{PRED_DIR}/{machine_type}_pred.csv'
    score_out_path = f'{SCORE_DIR}/{machine_type}_pred.csv'
    # save
    pd.to_pickle(output_dict, feature_out_path)
    torch.save(model.state_dict(), model_out_path)
    pred_df.to_csv(pred_out_path)
    score_df.to_csv(score_out_path)
    logger.info(f'SAVE SUCCESS : {model_out_path}')
    
    com.toc()

In [13]:
#run(machine_types[0], dev_paths)

In [ ]:
for machine_type in machine_types:
    run(machine_type, dev_paths)

2021-10-22 13:38:49,338 - 00_train.py - INFO - TARGET MACHINE_TYPE: pump
2021-10-22 13:38:49,339 - 00_train.py - INFO - MAKE DATA_LOADER
2021-10-22 13:38:49,344 - 00_train.py - INFO - TRAINING


use: cuda:0


100%|██████████| 600/600 [01:19<00:00,  7.53it/s]
2021-10-22 13:41:47,907 - pytorch_modeler.py - INFO - epoch:1/100, tr_loss:0.091607, src_loss:0.117892, src_mean_auc:0.716133, tgt_loss:0.113125, tgt_mean_auc:0.679167, mean_auc:0.697650,


,AUC,pAUC
Source_0,0.726300,0.591053
Source_1,0.841300,0.651053
Source_2,0.580800,0.563684
Target_0,0.804100,0.611579
Target_1,0.705900,0.604211
Target_2,0.527500,0.498421
mean,0.697650,0.586667
h_mean,0.678485,0.582614


100%|██████████| 45/45 [00:48<00:00,  1.07s/it]
2021-10-22 13:42:36,302 - pytorch_modeler.py - INFO - epoch:2/100, tr_loss:0.056927, src_loss:0.117892, src_mean_auc:0.716133, tgt_loss:0.113125, tgt_mean_auc:0.679167, mean_auc:0.697650,


...

100%|██████████| 45/45 [00:45<00:00,  1.02s/it]
2021-10-22 18:04:10,748 - pytorch_modeler.py - INFO - epoch:28/100, tr_loss:0.005373, src_loss:0.046635, src_mean_auc:0.778308, tgt_loss:0.150389, tgt_mean_auc:0.591120, mean_auc:0.684714,


,AUC,pAUC
Source_0,0.808400,0.687368
Source_1,0.696400,0.566842
Source_2,0.830125,0.812834
Target_0,0.726000,0.529474
Target_1,0.516122,0.540494
Target_2,0.531238,0.521034
mean,0.684714,0.609674
h_mean,0.661162,0.593628


100%|██████████| 45/45 [00:45<00:00,  1.02s/it]
2021-10-22 18:04:56,569 - pytorch_modeler.py - INFO - epoch:29/100, tr_loss:0.005164, src_loss:0.046635, src_mean_auc:0.778308, tgt_loss:0.150389, tgt_mean_auc:0.591120, mean_auc:0.684714,


,AUC,pAUC
Source_0,0.808400,0.687368
Source_1,0.696400,0.566842
Source_2,0.830125,0.812834
Target_0,0.726000,0.529474
Target_1,0.516122,0.540494
Target_2,0.531238,0.521034
mean,0.684714,0.609674
h_mean,0.661162,0.593628


  5%|▍         | 28/602 [00:04<01:32,  6.20it/s]IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

 34%|███▎      | 202/602 [00:29<00:49,  8.12it/s]

In [ ]:
#run(machine_types[0], dev_paths)

# run